<a href="https://colab.research.google.com/github/myy04/ResNet-CIFAR10/blob/main/hw2_2_cifar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ELEC4542 - Homework 2

In this notebook, we will walk through how to build and train a CNN to for image classification.

We will use PyTorch and torchvision to build and train our model.

Now, take a deep breath, prepare yourself, as we are about to get your hands dirty!

**You are expected to complete every code blocks marked with `TODO`.**

In [11]:
import torch
import torchvision
from torch import optim, nn, Tensor
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2

from __future__ import annotations

## Part 2: CIFAR-10 image classification

Nicely done!

We have built a simple CNN model and trained it on MNIST dataset.

Now, let's move on to a more challenging dataset - CIFAR-10.

CIFAR-10 is a dataset of 32x32 color images of 10 classes (airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck).
It has 50,000 training images and 10,000 test images.

In this part, we will be using an advanced CNN model (ResNet-18) taught in the lecture to classify CIFAR-10 images.

Try your best to achieve the best performance on CIFAR-10 dataset!
Note that your mark will NOT based on the performance of your model.
Instead, it will be based on the training process.

Have fun!

In [12]:
mean, std = (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)

transform = v2.Compose(
    [
        #############################################################################
        # TODO:                                                                     #
        # 1. update mean and std given the statistics of training data              #
        # 2. set your own data augmentation combinations                            #
        #############################################################################

        v2.RandomCrop(32, padding = 4, padding_mode = "reflect"),
        v2.RandomHorizontalFlip(p = 0.5),
        v2.ColorJitter(brightness = 0.1,contrast = 0.1,saturation = 0.1),
        v2.RandomAdjustSharpness(sharpness_factor = 2,p = 0.2),


        v2.ToTensor(),
        v2.Normalize(mean, std),
        #############################################################################
        #                          END OF YOUR CODE                                 #
        #############################################################################
    ]
)

train_dataset = datasets.CIFAR10("data", transform=transform, train=True, download=True)
test_dataset = datasets.CIFAR10("data", transform=transform, train=False, download=True)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

#############################################################################
# TODO:                                                                     #
# 1. Use a pre-defined model in torchvision.models                          #
# 2. Define the criterion                                                   #
# 3. Define the optimizer                                                   #
# 4. Adjust the hyperparameters                                             #
#############################################################################

model = torchvision.models.resnet18(pretrained = False)
model.fc = nn.Linear(model.fc.in_features, 10)
device = torch.device('cuda')
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 1e-3, momentum=0.7, weight_decay=5e-7)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max = 20, eta_min=1e-5)


#############################################################################
#                          END OF YOUR CODE                                 #
#############################################################################

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [13]:
def train_step(input: Tensor, label: Tensor):
    #############################################################################
    # TODO: implement a training step                                           #
    #############################################################################

    model.train()

    input = input.to(device)
    label = label.to(device)

    output = model(input)
    loss = criterion(output, label)

    optimizer.zero_grad()
    loss.backward()

    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    # scheduler.step()

    return loss, output


@torch.no_grad()
def test_step(input: Tensor, label: Tensor):
    #############################################################################
    # TODO: implement a testing step                                           #
    #############################################################################

    model.eval()

    input = input.to(device)
    label = label.to(device)

    output = model(input)
    loss = criterion(output, label)

    return loss, output


def train_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = train_step(input, label)

        #############################################################################
        # TODO: implement accuracy calculation                                      #
        # You may add additional metrics to better evaluate your model              #
        #############################################################################

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        correct += (predicted == label.to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")


    scheduler.step()
    print(f"Train Accuracy: {correct / samples * 100.00}%")

        #############################################################################
        #                          END OF YOUR CODE                                 #
        #############################################################################



def test_epoch(dataloader):
    correct = 0
    samples = 0

    for i, (input, label) in enumerate(dataloader):
        loss, output = test_step(input, label)
        #############################################################################
        # TODO: implement accuracy calculation                                      #
        # You may add additional metrics to better evaluate your model              #
        #############################################################################

        _, predicted = torch.max(output.data, 1)

        samples += label.size(0)
        correct += (predicted == label.to(device)).sum().item()

        if i % 100 == 0:
            print(f"Loss [{i}/{len(dataloader)}]: {loss.item()}")

    print(f"Test Accuracy: {correct / samples * 100.00}%")

        #############################################################################
        #                          END OF YOUR CODE                                 #
        #############################################################################


In [14]:
for epoch in range(20):
    print(f"epoch: {epoch}")
    train_epoch(train_dataloader)
    test_epoch(test_dataloader)

epoch: 0
Loss [0/6250]: 2.9934277534484863
Loss [100/6250]: 3.089221477508545
Loss [200/6250]: 2.036905527114868
Loss [300/6250]: 2.485729694366455
Loss [400/6250]: 2.6440584659576416
Loss [500/6250]: 2.4041454792022705
Loss [600/6250]: 2.0921390056610107
Loss [700/6250]: 2.1698484420776367
Loss [800/6250]: 1.545264720916748
Loss [900/6250]: 2.3604073524475098
Loss [1000/6250]: 1.9104199409484863
Loss [1100/6250]: 1.5051521062850952
Loss [1200/6250]: 2.085587501525879
Loss [1300/6250]: 1.6903482675552368
Loss [1400/6250]: 1.967579960823059
Loss [1500/6250]: 2.110064744949341
Loss [1600/6250]: 1.7403547763824463
Loss [1700/6250]: 2.2502613067626953
Loss [1800/6250]: 2.087735652923584
Loss [1900/6250]: 1.560302734375
Loss [2000/6250]: 1.9601447582244873
Loss [2100/6250]: 2.2211320400238037
Loss [2200/6250]: 1.3972539901733398
Loss [2300/6250]: 1.4767810106277466
Loss [2400/6250]: 2.3401336669921875
Loss [2500/6250]: 2.0098178386688232
Loss [2600/6250]: 1.6433379650115967
Loss [2700/6250]

Is your model working well on CIFAR-10 dataset?

Try to use a different optimizer, learning rate scheduler, data augmentation, etc. to improve the performance of your model.
Alternatively, you can try to use a different model (e.g. ResNet-34, ResNet-50, etc.) to achieve better performance.